In [1]:
import torch 
import torch.nn.functional as F 
from torch import Tensor 
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import re

In [2]:
# Load csv File 

df = pd.read_csv('/home/pedro/Documents/JADS/DataConsul/jads_data.csv')

/tmp/ipykernel_30904/2859765203.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/pedro/Documents/JADS/DataConsul/jads_data.csv')


In [3]:
# Take product title and description

product_titles = df['title'].tolist()
product_descriptions = df['description'].tolist()

In [4]:
def last_token_pool(last_hidden_states: Tensor,
                   attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding: 
        return last_hidden_states[:, -1]
    else: 
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
    
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

In [5]:
# Each query must come with a one-sentence instruction that describes the task

task = 'Given a web search query, retrieve relevant passages that answer the query'

queries = [
    get_detailed_instruct(task, 'How to bake a chocolate cake'),
    get_detailed_instruct(task, 'Symptoms of the flu') 
]

passages = [
    "To bake a delicious chocolate cake, you'll need the following ingredients: all-purpose flour, sugar, cocoa powder, baking powder, baking soda, salt, eggs, milk, vegetable oil, and vanilla extract. Start by preheating your oven to 350°F (175°C). In a mixing bowl, combine the dry ingredients (flour, sugar, cocoa powder, baking powder, baking soda, and salt). In a separate bowl, whisk together the wet ingredients (eggs, milk, vegetable oil, and vanilla extract). Gradually add the wet mixture to the dry ingredients, stirring until well combined. Pour the batter into a greased cake pan and bake for 30-35 minutes. Let it cool before frosting with your favorite chocolate frosting. Enjoy your homemade chocolate cake!",
    "The flu, or influenza, is an illness caused by influenza viruses. Common symptoms of the flu include a high fever, chills, cough, sore throat, runny or stuffy nose, body aches, headache, fatigue, and sometimes nausea and vomiting. These symptoms can come on suddenly and are usually more severe than the common cold. It's important to get plenty of rest, stay hydrated, and consult a healthcare professional if you suspect you have the flu. In some cases, antiviral medications can help alleviate symptoms and reduce the duration of the illness."
]

In [6]:
# Load the model and the tokenizer

tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

: 

In [ ]:
# Get the embeddings 

max_length = 4096 
# input_texts = product_titles + product_descriptions 
input_texts = product_descriptions 

batch_size = 32

In [ ]:
for i in range(0, len(input_texts), batch_size):
    batch_texts = input_texts[i:i+batch_size]
    batch_dict = tokenizer(batch_texts, max_length=max_length, padding=True, truncation=True, return_tensores="pt")
    outputs = model(**batch_dict)
    embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    print(embeddings)

In [ ]:
# #Normalize embeddings

# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist)